# Initialization

In [4]:
import numpy as np
from pprint import pprint


from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
#from laboneq.contrib.example_helpers.generate_device_setup import generate_device_setup
from laboneq.dsl.device import DeviceSetup




descriptor = generate_descriptor(
    #pqsc=[""], # 장비 여러개 사용시
    shfqc_6=["DEV12256"],
    number_data_qubits=3,
    multiplex=True,
    number_multiplex=3,
    include_cr_lines=True,
    include_ef_lines=True,
    get_zsync=False,  # Only set to True when using real device
    save = True,
    filename="JW_KAIST",
    ip_address="192.168.0.83"
)


#descriptor
setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="192.168.0.83")
#setup
setup.instruments[0].device_options = 'SHFQC/PLUS/QC6CH'
#setup.instruments

In [5]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=True, do_emulation=False)
#session.disconnect()

[2025.09.08 19:57:54.926] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/yalgaeahn/JSAHN/qubit-experiment/examples/laboneq_output/log
[2025.09.08 19:57:54.929] INFO    VERSION: laboneq 2.58.0
[2025.09.08 19:57:54.929] INFO    Connecting to data server at 192.168.0.83:8004
[2025.09.08 19:57:54.967] INFO    Connected to Zurich Instruments LabOne Data Server version 25.07.0.507 at 192.168.0.83:8004
[2025.09.08 19:57:55.026] INFO    Configuring the device setup
[2025.09.08 19:57:55.053] INFO    The device setup is configured


In [6]:
from qpu_types import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters
from operations import TransmonOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform



#_loaded_qubits = load_qubit_parameters(filename="latest") 
qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 
#qubits = list(_loaded_qubits.values())

qubits = []
for i in range(3):
    q = TransmonQubit.from_logical_signal_group(
        f"q{i}",
        setup.logical_signal_groups[f"q{i}"],
        parameters=TransmonQubitParameters()
    )
    qubits.append(q)



qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
#quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)

# qpu.topology.add_edge("bus", "q16", "q17")
# qpu.topology.add_edge("bus", "q17", "q18")
# qpu.topology.add_edge("bus", "q18", "q19")
# qpu.topology.plot()




from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


BF 7

In [100]:
# 0 : 15 , 1 : 16 , 2 : 17, 3 : 18, 4 : 19
for q in qubits:
    #q.parameters.ge_drive_pulse["sigma"] = 0.25
    q.parameters.readout_amplitude = 1.0
    q.parameters.reset_delay_length = 20e-6

    q.parameters.readout_range_out = -10
    q.parameters.readout_range_in = 0
    q.parameters.drive_range = 10

    q.parameters.readout_lo_frequency = 7.6e9
    q.parameters.drive_lo_frequency = 4.8e9

    q.parameters.spectroscopy_length = 10e-6

    q.parameters.ge_drive_length =64e-9
    q.parameters.readout_integration_delay = None#####

 
qubits[0].parameters.resonance_frequency_ge = 4.8048e9
qubits[0].parameters.readout_resonator_frequency = 7.6267e9

qubits[1].parameters.resonance_frequency_ge = 4.5615e9
qubits[1].parameters.readout_resonator_frequency = 7.3094e9
# qubits[2].parameters.resonance_frequency_ge = 4.716e9
# qubits[3].parameters.resonance_frequency_ge = 4.664e9
# qubits[4].parameters.resonance_frequency_ge = 4.779e9

# qubits[2].parameters.readout_resonator_frequency = 6.9226e9
# qubits[3].parameters.readout_resonator_frequency = 6.8433e9
# qubits[4].parameters.readout_resonator_frequency = 6.7739e9



3 oscillator to be calibrated
- `drive_local_osc`
- `readout_local_osc`
- `readout_acquire_osc`

In [5]:
from laboneq.dsl.calibration import Calibration, Oscillator, SignalCalibration

sibal= SignalCalibration()
sibal.delay_signal

In [6]:
from experiments import signal_propagation_delay
options = signal_propagation_delay.experiment_workflow.options()
#workflow.show_fields(options)

Correcting propagation delay. 

$\Delta \phi =-2\pi f t$ $\implies$ $\tau = \frac{1}{\Delta f}$

In [ ]:
from experiments import signal_propagation_delay
####################################################################################
qubits[0].parameters.readout_amplitude =1.0
q=qubits[0]
delays=np.linspace(0e-9, 200e9, 51)
measure_delays = 40e-6
pprint(q.readout_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#####################################################################################
options = signal_propagation_delay.experiment_workflow.options()
#workflow.show_fields(options)
options.update(False)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
prop_delay = signal_propagation_delay.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    delays=delays,
    measure_delay=measure_delays,
    options=options
)
prop_delay_result=prop_delay.run()
prop_delay_analysis_result = prop_delay_result.tasks["analysis_workflow"]
pprint(prop_delay_analysis_result.output)

In [ ]:
prop_delay_analysis_result.tasks["plot_data"].output['q0']

# 1. Spectroscopy

In [97]:

qubits[1].parameters.drive_range = 10
qubits[1].parameters.readout_range_out= -10

In [ ]:
from helper import adjust_amplitude_for_output_range, calculate_power
adjust_amplitude_for_output_range(initial_output_dbm=0, initial_amplitude= 0.002, new_output_dbm=-5)
calculate_power(output_dbm=0, amplitude=0.001,attenuation=[])

### 1.1 Resonator spectroscopy with amplitude sweep

STRATEGY
1. CW spectroscpy for wide range scan
    * CW spectroscopy can be done for wide range scan 
    * amplitude, frequency 2D sweep 을 통해  power에 따른 broadening 및 SNR 경향 관찰
    * $\Delta F$ : frequency span,  $\Delta f_r$
    * Pick $\Delta F / \Delta f_r$ 1~2

In [101]:
from experiments import resonator_spectroscopy_amplitude

####################################################################################
qubits[1].parameters.readout_amplitude =1.0
q=qubits[1]
pprint(q.readout_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-5e6,5e6,101)
amplitudes = np.linspace(0.0, 1.0, 21) #This works as scaling factor!!!!!!!!!!!!
#####################################################################################
options = resonator_spectroscopy_amplitude.experiment_workflow.options()
#workflow.show_fields(options)
options.update(False)
options.use_cw(False)
options.do_analysis(True)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
res_spec_amp = resonator_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)
res_spec_amp_result=res_spec_amp.run()


('measure',
 {'amplitude': 1.0,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
DRIVE range (dBm) : 10, READOUT range (dBm) : -10
[2025.09.08 21:03:31.055] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:03:31.056] INFO      Workflow 'resonator_spectroscopy_amplitude': execution started at 2025-09-08  
[2025.09.08 21:03:31.056] INFO      12:03:31.055045Z                                                              
[2025.09.08 21:03:31.056] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:03:31.068] INFO    Task 'temporary_qpu': started at 2025-09-08 12:03:31.068738Z
[2025.09.08 21:03:31.069] INFO    Task 'temporary_qpu': ended at 2025-09-08 12:03:31.069731Z
[2025.09.08 21:03:31.072] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-08 
[2025.09.08 21:03:31.072] INFO    12:03

In [ ]:
res_spec_amp_result.tasks["analysis_workflow"].tasks["plot_signal_magnitude_and_phase_2d"].output["q0"]

### 1.2 Resonator Spectroscopy
* CW not recommended 

In [8]:
qubits[0].parameters.readout_integration_delay 
qubits[0].parameters.readout_integration_kernels_type

'default'

In [ ]:
qubits[0].readout_integration_parameters()

In [105]:
from experiments import resonator_spectroscopy

####################################################################################
q=qubits[1]
q.parameters.readout_amplitude = 0.5
print(q.readout_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#frequencies = q.parameters.readout_resonator_frequency + np.linspace(-10e6,10e6,501)
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-10e6,10e6,1001)
#####################################################################################
options = resonator_spectroscopy.experiment_workflow.options()
#workflow.show_fields(options)
options.count(1024)
options.update(True)
options.use_cw(False) #qop.measure 
options.fit_complex_resonator(False)
options.fit_lorentzian(True)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
res_spec = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    options=options
)
res_spec_result = res_spec.run()
res_spec_analysis_result = res_spec_result.tasks["analysis_workflow"]
pprint(res_spec_analysis_result.output)

('measure', {'amplitude': 0.5, 'length': 2e-06, 'pulse': {'function': 'GaussianSquare', 'sigma': 0.2, 'risefall_sigma_ratio': 3.0}})
DRIVE range (dBm) : 10, READOUT range (dBm) : -10
[2025.09.08 21:05:31.256] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:05:31.257] INFO      Workflow 'resonator_spectroscopy': execution started at 2025-09-08            
[2025.09.08 21:05:31.257] INFO      12:05:31.256365Z                                                              
[2025.09.08 21:05:31.258] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:05:31.274] INFO    Task 'temporary_qpu': started at 2025-09-08 12:05:31.273880Z
[2025.09.08 21:05:31.275] INFO    Task 'temporary_qpu': ended at 2025-09-08 12:05:31.275591Z
[2025.09.08 21:05:31.280] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-08 
[2025.09.08 21:05:31.282] INFO    12:05:31.279028Z
[2025.09.08 21:05

In [ ]:
res_spec_analysis_result.tasks['plot_magnitude_phase'].output

In [ ]:
# res_spec_result.tasks["analysis_workflow"].tasks["plot_real_imaginary"].output

In [ ]:
from laboneq.simple import show_pulse_sheet
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

#show_pulse_sheet(compiled_experiment=res_spec_result.tasks["compile_experiment"].output,name='test', interactive=True)
#plot_simulation(compiled_experiment=res_spec_result.tasks["compile_experiment"].output)

### 2.1 Qubit Spectroscopy with Amplitude Sweep
* Current qubit spectroscopy modules do not support CW 
* Recommended to use long enough spectroscopy drive (pulsed) to see steady state signals

In [106]:
from experiments import qubit_spectroscopy_amplitude

###############################################################
q = qubits[1]
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-2e6, 2e6, 101)
amplitudes = np.linspace(0.01, 1.0, 11)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")

pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = qubit_spectroscopy_amplitude.experiment_workflow.options()
options.update(False)
# workflow.show_fields(options)
###################################################################
qubit_spec_amp = qubit_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)

qubit_spec_amp_result = qubit_spec_amp.run()
qubit_spec_amp_result

DRIVE range (dBm) : 10, READOUT range (dBm) : -10
('measure',
 {'amplitude': 0.5,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
('drive',
 {'amplitude': 1,
  'length': 1e-05,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.09.08 21:06:10.442] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:06:10.443] INFO      Workflow 'qubit_spectroscopy_amplitude': execution started at 2025-09-08      
[2025.09.08 21:06:10.443] INFO      12:06:10.440679Z                                                              
[2025.09.08 21:06:10.444] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:06:10.459] INFO    Task 'temporary_qpu': started at 2025-09-08 12:06:10.458822Z
[2025.09.08 21:06:10.460] INFO    Task 'temporary_qpu': ended at 2025-09-08

WorkflowResult(name=qubit_spectroscopy_amplitude, index=())

### 2.2 Qubit Spectroscopy 


In [108]:
from experiments import qubit_spectroscopy


###############################################################
q = qubits[1]
q.parameters.spectroscopy_amplitude = 1.0
q.parameters.readout_amplitude = 1.0
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-3e6, 3e6, 101)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = qubit_spectroscopy.experiment_workflow.options()
options.update(True)
options.count(1024*2)
# workflow.show_fields(options)
###################################################################
qubit_spec = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    options=options
)

qubit_spec_result = qubit_spec.run()
qubit_spec_result

DRIVE range (dBm) : 10, READOUT range (dBm) : -10
('measure',
 {'amplitude': 1.0,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
('drive',
 {'amplitude': 1.0,
  'length': 1e-05,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.09.08 21:08:03.511] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:08:03.512] INFO      Workflow 'qubit_spectroscopy': execution started at 2025-09-08                
[2025.09.08 21:08:03.513] INFO      12:08:03.509122Z                                                              
[2025.09.08 21:08:03.514] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:08:03.529] INFO    Task 'temporary_qpu': started at 2025-09-08 12:08:03.529358Z
[2025.09.08 21:08:03.530] INFO    Task 'temporary_qpu': ended at 2025-09-

WorkflowResult(name=qubit_spectroscopy, index=())

# 2. Rabi

## 2.1 Rabi Amplitude 

In [109]:
qubits[1].parameters.ge_drive_amplitude_pi = 1.0

In [110]:
qubits[1].parameters.drive_range = 10

In [113]:
from laboneq_applications.experiments import amplitude_rabi
q = qubits[1]

#######################################################################
amplitudes = np.linspace(0.02, 1.0,101)
print(q.parameters.ge_drive_amplitude_pi)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = amplitude_rabi.experiment_workflow.options()
options.update(True)
options.use_cal_traces(True)
options.do_pca(False)
#print(workflow.show_fields(options))

###################################################################
rabi_amp = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    amplitudes=amplitudes,
    options=options
)

rabi_amp_result = rabi_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_result.tasks['analysis_workflow'].output)

0.5921856168385499
DRIVE range (dBm) : 10, READOUT range (dBm) : -10
[2025.09.08 21:10:07.855] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:10:07.856] INFO      Workflow 'amplitude_rabi': execution started at 2025-09-08 12:10:07.855542Z   
[2025.09.08 21:10:07.856] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:10:07.870] INFO    Task 'temporary_qpu': started at 2025-09-08 12:10:07.870429Z
[2025.09.08 21:10:07.871] INFO    Task 'temporary_qpu': ended at 2025-09-08 12:10:07.871510Z
[2025.09.08 21:10:07.874] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-08 
[2025.09.08 21:10:07.874] INFO    12:10:07.874063Z
[2025.09.08 21:10:07.875] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-08 12:10:07.875180Z
[2025.09.08 21:10:07.876] INFO    Task 'create_experiment': started at 2025-09-08 12:10:07.876205Z
[2025.09.08 21:10:07.881] INFO  

## 2.2 (optional) Rabi Amplitude Chevron 

In [80]:
from experiments import amplitude_rabi_chevron

#######################################################################
q = qubits[0]
amplitudes = np.linspace(0,1.0,11)
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-3e6, 3e6, 51)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = amplitude_rabi_chevron.experiment_workflow.options()

#print(workflow.show_fields(options))

###################################################################
rabi_amp_chev = amplitude_rabi_chevron.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)

rabi_amp_chev_result = rabi_amp_chev.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_chev_result.tasks['analysis_workflow'].output)


DRIVE range (dBm) : 10, READOUT range (dBm) : -10
[2025.09.08 20:40:05.369] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 20:40:05.371] INFO      Workflow 'amplitude_rabi_chevron': execution started at 2025-09-08            
[2025.09.08 20:40:05.372] INFO      11:40:05.369570Z                                                              
[2025.09.08 20:40:05.372] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 20:40:05.396] INFO    Task 'create_experiment': started at 2025-09-08 11:40:05.394112Z
[2025.09.08 20:40:05.420] INFO    Task 'create_experiment': ended at 2025-09-08 11:40:05.420420Z
[2025.09.08 20:40:05.422] INFO    Task 'compile_experiment': started at 2025-09-08 11:40:05.422364Z
[2025.09.08 20:40:05.425] INFO    Resolved modulation type of oscillator 'q0_readout_acquire_osc' on signal 'q0/acquire' to SOFTWARE
[2025.09.08 20:40:05.427] INFO    Resolved modulation type of

## 2.3 (optional) Rabi Length
- Not working yet

In [ ]:
# from experiments import time_rabi
# #######################################################################
# q = qubits[2]
# q.parameters.ge_drive_amplitude_pi = 0.3
# lengths = np.linspace(0.0, 240e-9, 21 ),
# print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
# #######################################################################
# options = time_rabi.experiment_workflow.options()

# #print(workflow.show_fields(options))

# ###################################################################
# rabi_length = time_rabi.experiment_workflow(
#     session=session,
#     qpu=qpu,
#     qubits=q,
#     lengths=lengths,
#     options=options
# )

# rabi_length_result = rabi_length.run()
# #qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
# print(rabi_length_result.tasks['analysis_workflow'].output)


## 2.2 (optional) Rabi Length Chevron

In [ ]:
# from experiments import time_rabi_chevron
# #######################################################################
# q = qubits[4]

# frequencies = q.parameters.resonance_frequency_ge + np.linspace(-5e6, 5e6, 21)
# lengths = np.linspace(0.0, 240e-9, 21 ),
# print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
# #######################################################################
# options = time_rabi_chevron.experiment_workflow.options()

# #print(workflow.show_fields(options))

# ###################################################################
# rabi_length_chev = time_rabi_chevron.experiment_workflow(
#     session=session,
#     qpu=qpu,
#     qubits=q,
#     frequencies=frequencies,
#     lengths=lengths,
#     options=options
# )

# rabi_length_chev_result = rabi_length_chev.run()
# #qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
# print(rabi_length_chev_result.tasks['analysis_workflow'].output)


# 3. Ramsey

In [117]:
from experiments import ramsey

q = qubits[1]
#######################################################################
delays = np.linspace(0,6e-6,301)
detunings = 0.7e6
pprint(q.readout_parameters())
#######################################################################
options = ramsey.experiment_workflow.options()
options.update(True)
options.count(1024)
options.use_cal_traces(True)
# Build and run Ramsey workflow (no explicit command table options required)
ramsey_wf = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    options=options
)
ramsey_result = ramsey_wf.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(ramsey_result.tasks['analysis_workflow'].output)

('measure',
 {'amplitude': 1.0,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.09.08 21:11:51.800] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:11:51.800] INFO      Workflow 'ramsey': execution started at 2025-09-08 12:11:51.799151Z           
[2025.09.08 21:11:51.801] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 21:11:51.815] INFO    Task 'temporary_qpu': started at 2025-09-08 12:11:51.815469Z
[2025.09.08 21:11:51.818] INFO    Task 'temporary_qpu': ended at 2025-09-08 12:11:51.817881Z
[2025.09.08 21:11:51.820] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-08 
[2025.09.08 21:11:51.821] INFO    12:11:51.820670Z
[2025.09.08 21:11:51.822] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-08 12:11:51.821686Z
[2025.09.08 21:11:51.823] INFO    Task

In [ ]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[4]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
workflow.show_fields(options)

In [87]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[0]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,20)
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
options.update(True)
options.use_cal_traces(True)
options.do_pca(False)

#print(workflow.show_fields(options))

###################################################################
error_amp = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_result = error_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_result.tasks['analysis_workflow'].output)

[2025.09.08 20:46:55.718] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 20:46:55.719] INFO      Workflow 'experiment_workflow_x180': execution started at 2025-09-08          
[2025.09.08 20:46:55.720] INFO      11:46:55.718410Z                                                              
[2025.09.08 20:46:55.720] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 20:46:55.725] WARNING Type <class 'dict'> has unsupported keys or values. Keys must be strings and values must have one of the following types: [NoneType, int, float, complex, bool, str, dict, list, tuple, integer, ndarray] [name: experiment_workflow_x180.input.temporary_parameters].
[2025.09.08 20:46:55.735] INFO    Task 'temporary_qpu': started at 2025-09-08 11:46:55.735364Z
[2025.09.08 20:46:55.736] WARNING Type <class 'dict'> has unsupported keys or values. Keys must be strings and values must have one of the following

In [89]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[0]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,20)
#######################################################################
options = amplitude_fine.experiment_workflow_x90.options()
options.update(True)
options.use_cal_traces(True)
options.do_pca(False)

#print(workflow.show_fields(options))

###################################################################
error_amp_half = amplitude_fine.experiment_workflow_x90(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_half_result = error_amp_half.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_half_result.tasks['analysis_workflow'].output)

[2025.09.08 20:48:22.306] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 20:48:22.306] INFO      Workflow 'experiment_workflow_x90': execution started at 2025-09-08           
[2025.09.08 20:48:22.307] INFO      11:48:22.305682Z                                                              
[2025.09.08 20:48:22.307] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 20:48:22.313] WARNING Type <class 'dict'> has unsupported keys or values. Keys must be strings and values must have one of the following types: [NoneType, int, float, complex, bool, str, dict, list, tuple, integer, ndarray] [name: experiment_workflow_x90.input.temporary_parameters].
[2025.09.08 20:48:22.326] INFO    Task 'temporary_qpu': started at 2025-09-08 11:48:22.326143Z
[2025.09.08 20:48:22.326] WARNING Type <class 'dict'> has unsupported keys or values. Keys must be strings and values must have one of the following 

In [92]:
from laboneq_applications.experiments import drag_q_scaling
q = qubits[0]
temporary_parameters = q.parameters.copy()

#######################################################################
q_scalings = np.linspace(-0.1,0.1,51)
#######################################################################
options = drag_q_scaling.experiment_workflow.options()
options.update(True)



#print(workflow.show_fields(options))

###################################################################
drag = drag_q_scaling.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    q_scalings=q_scalings,
    options=options
)

drag_result = drag.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(drag_result.tasks['analysis_workflow'].output)

[2025.09.08 20:51:42.385] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 20:51:42.386] INFO      Workflow 'drag_q_scaling': execution started at 2025-09-08 11:51:42.382884Z   
[2025.09.08 20:51:42.387] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.08 20:51:42.411] INFO    Task 'temporary_qpu': started at 2025-09-08 11:51:42.410380Z
[2025.09.08 20:51:42.415] INFO    Task 'temporary_qpu': ended at 2025-09-08 11:51:42.413052Z
[2025.09.08 20:51:42.459] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-08 
[2025.09.08 20:51:42.460] INFO    11:51:42.458814Z
[2025.09.08 20:51:42.461] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-08 11:51:42.461028Z
[2025.09.08 20:51:42.463] INFO    Task 'create_experiment': started at 2025-09-08 11:51:42.463029Z
[2025.09.08 20:51:42.475] INFO    Task 'create_experiment': ended at 2025-09-08 11:51:42.475158Z
[202

# Basic Readout optimization
Here we optimize SNR via

* `readout_length`
* `readout-amplitude`

* `readout_integration_length`
* `readout_integration_delay`
* `readout_integration_weight`

In [ ]:
from experiments import dispersive_shift

In [ ]:
from experiments import time_traces

# Save Qubit Parameters

In [118]:
from helper import load_qubit_parameters, save_qubit_parameters
save_qubit_parameters(qubits={ qubit.uid : qubit for qubit in qubits}, save_folder="./qubit_parameters", filename="KAIST_TG")

Qubit parameters stored as JSON in /Users/yalgaeahn/JSAHN/qubit-experiment/examples/qubit_parameters/20250908-2119_KAIST_TG.json
